In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression

np.sort(os.listdir('./subs'))

array(['oof_0.9235726706987148.csv', 'oof_cat_0.9180511510872795.csv',
       'oof_lgb_0.9160928879646695.csv', 'oof_lgb_0.9224806732079125.csv',
       'oof_lgb_0.9225898803415156.csv', 'oof_lgb_0.9229.csv',
       'oof_lgb_0.92342.csv', 'oof_lgb_0.9237994961308414.csv',
       'oof_lgb_0.923864316198583.csv', 'oof_xgb_0.9175107247524672.csv',
       'sub_0.9235726706987148.csv', 'sub_cat_0.9180511510872795.csv',
       'sub_lgb_0.9160928879646695.csv', 'sub_lgb_0.9224806732079125.csv',
       'sub_lgb_0.9225898803415156.csv', 'sub_lgb_0.9229.csv',
       'sub_lgb_0.92342.csv', 'sub_lgb_0.9237994961308414.csv',
       'sub_lgb_0.923864316198583.csv', 'sub_xgb_0.9175107247524672.csv'],
      dtype='<U30')

In [2]:
pred = list(np.sort([c for c in (os.listdir('./subs')) if 'sub' in c]))

In [3]:
pred

['sub_0.9235726706987148.csv',
 'sub_cat_0.9180511510872795.csv',
 'sub_lgb_0.9160928879646695.csv',
 'sub_lgb_0.9224806732079125.csv',
 'sub_lgb_0.9225898803415156.csv',
 'sub_lgb_0.9229.csv',
 'sub_lgb_0.92342.csv',
 'sub_lgb_0.9237994961308414.csv',
 'sub_lgb_0.923864316198583.csv',
 'sub_xgb_0.9175107247524672.csv']

In [4]:
oof = list(np.sort([c for c in (os.listdir('./subs')) if 'oof' in c]))

In [5]:
target = pd.read_csv('./input/train.csv')['target']

In [6]:
val = pd.DataFrame()
for j,i in enumerate(oof):
    val['m{}'.format(j)] = pd.read_csv(os.path.join('./subs',i))['oof']
    print(i,'\t auc : ', roc_auc_score(target,val['m{}'.format(j)]))

oof_0.9235726706987148.csv 	 auc :  0.9235726706987148
oof_cat_0.9180511510872795.csv 	 auc :  0.9180511510872796
oof_lgb_0.9160928879646695.csv 	 auc :  0.9160928879646695
oof_lgb_0.9224806732079125.csv 	 auc :  0.9224806732079125
oof_lgb_0.9225898803415156.csv 	 auc :  0.9225898803415156
oof_lgb_0.9229.csv 	 auc :  0.9229102322743912
oof_lgb_0.92342.csv 	 auc :  0.9234290920692649
oof_lgb_0.9237994961308414.csv 	 auc :  0.9237994961308414
oof_lgb_0.923864316198583.csv 	 auc :  0.923864316198583
oof_xgb_0.9175107247524672.csv 	 auc :  0.9175107247524672


In [7]:
val.corr()

,m0,m1,m2,m3,m4,m5,m6,m7,m8,m9
m0,1.000000,0.976275,0.973499,0.987629,0.989253,0.988651,0.979474,0.980428,0.979630,0.976052
m1,0.976275,1.000000,0.952891,0.973310,0.974209,0.973836,0.956688,0.957289,0.956413,0.964009
m2,0.973499,0.952891,1.000000,0.966279,0.966821,0.961334,0.949708,0.950803,0.949619,0.956455
m3,0.987629,0.973310,0.966279,1.000000,0.996484,0.990229,0.975875,0.976782,0.975837,0.971604
m4,0.989253,0.974209,0.966821,0.996484,1.000000,0.991869,0.979091,0.980040,0.979056,0.972762
m5,0.988651,0.973836,0.961334,0.990229,0.991869,1.000000,0.983864,0.985204,0.983990,0.972332
m6,0.979474,0.956688,0.949708,0.975875,0.979091,0.983864,1.000000,0.995840,0.996303,0.958573
m7,0.980428,0.957289,0.950803,0.976782,0.980040,0.985204,0.995840,1.000000,0.996710,0.959340
m8,0.979630,0.956413,0.949619,0.975837,0.979056,0.983990,0.996303,0.996710,1.000000,0.958197
m9,0.976052,0.964009,0.956455,0.971604,0.972762,0.972332,0.958573,0.959340,0.958197,1.000000


In [8]:
cv = cross_val_score(LinearRegression(), val, target, cv = 5, scoring = 'roc_auc')
print(cv.mean(), cv.std())

0.9245316314800366 0.002068595305655094


In [9]:
from sklearn.model_selection import StratifiedKFold

models = []
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(val))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(val.values, target.values)):

    X_tr = val.iloc[trn_idx]
    y_tr = target.iloc[trn_idx]
    
    X_val = val.iloc[val_idx]
    y_val = target.iloc[val_idx]
    model = LinearRegression()
    model.fit(X_tr, y_tr)
    models.append(model)
    oof[val_idx] = model.predict(X_val)
    print("Fold {}".format(fold_),'roc_auc_score : ', roc_auc_score(y_val,oof[val_idx]))
    
score = roc_auc_score(target,oof)
print('roc_auc_score : ', score)
     

Fold 0 roc_auc_score :  0.9234984232280692
Fold 1 roc_auc_score :  0.9234862069962022
Fold 2 roc_auc_score :  0.9293184508253618
Fold 3 roc_auc_score :  0.9241383583619829
Fold 4 roc_auc_score :  0.9219864412799625
roc_auc_score :  0.9244718016050045


In [10]:
test = pd.read_csv('./subs/'+pred[0])
test.rename(columns={'target':'m0'},inplace = True)
for j,i in enumerate(pred[1:]):
    df = pd.read_csv(os.path.join('./subs',i))
    df.rename(columns={'target':'m{}'.format(j+1)},inplace = True)
    test = test.merge(df, on='ID_code')
        
    

In [11]:
test.corr()

,m0,m1,m2,m3,m4,m5,m6,m7,m8,m9
m0,1.000000,0.990650,0.768680,0.993359,0.994468,0.994312,0.985356,0.984884,0.984758,0.993996
m1,0.990650,1.000000,0.760585,0.989072,0.989472,0.989305,0.973131,0.972206,0.972111,0.991001
m2,0.768680,0.760585,1.000000,0.759885,0.761297,0.760598,0.760832,0.760664,0.760563,0.766020
m3,0.993359,0.989072,0.759885,1.000000,0.999429,0.995873,0.981843,0.981131,0.980880,0.991019
m4,0.994468,0.989472,0.761297,0.999429,1.000000,0.996905,0.984529,0.983897,0.983677,0.991447
m5,0.994312,0.989305,0.760598,0.995873,0.996905,1.000000,0.989534,0.989031,0.988838,0.991499
m6,0.985356,0.973131,0.760832,0.981843,0.984529,0.989534,1.000000,0.999621,0.999607,0.978728
m7,0.984884,0.972206,0.760664,0.981131,0.983897,0.989031,0.999621,1.000000,0.999765,0.977965
m8,0.984758,0.972111,0.760563,0.980880,0.983677,0.988838,0.999607,0.999765,1.000000,0.977677
m9,0.993996,0.991001,0.766020,0.991019,0.991447,0.991499,0.978728,0.977965,0.977677,1.000000


In [12]:
test.head(20)

,ID_code,m0,m1,m2,m3,m4,m5,m6,m7,m8,m9
0,test_0,0.000000,0.000000,0.069807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,test_1,0.000000,0.000000,0.156410,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,test_2,0.000000,0.000000,0.195708,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,test_3,0.135143,0.110310,0.209002,0.122962,0.115048,0.087671,0.051844,0.054520,0.053801,0.096720
4,test_4,0.000000,0.000000,0.066254,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,test_5,0.000000,0.000000,0.001283,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,test_6,0.000000,0.000000,0.005188,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,test_7,0.060201,0.047764,0.078715,0.059908,0.061299,0.044704,0.026788,0.026168,0.026936,0.050600
8,test_8,0.000000,0.000000,0.001787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,test_9,0.000000,0.000000,0.005260,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
sub = pd.read_csv('./subs/sub_cat_0.9180511510872795.csv')

In [14]:
preds = 0.
for model in models:
    preds+=model.predict(test[[c for c in test.columns if c!='ID_code']])/5.

In [15]:
sub['target'] = preds

In [16]:
sub.head(10)

,ID_code,target
0,test_0,0.001111
1,test_1,0.001691
2,test_2,0.001954
3,test_3,0.098556
4,test_4,0.001088
5,test_5,0.000653
6,test_6,0.000679
7,test_7,0.045786
8,test_8,0.000656
9,test_9,0.000680


In [17]:
sub.to_csv('linear_stacking_{}.csv;'.format(score), index = False)